## NLP Project


## Task #1: Understand the problem

## Task #2: Import Libries

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

## Task #3: Load data

In [4]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
# relable
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Task #4:Clean and Prepare the Data

In [11]:
# create a new length feature 
from pyspark.sql.functions import length

In [12]:
data = data.withColumn('length',length(data['text']))

In [13]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
# hw many the avg of length ham and spam
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Task #5: Feature Transformations


In [16]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [18]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

## Task #6: The Model



In [22]:
from pyspark.ml.classification import NaiveBayes

In [23]:
nb = NaiveBayes()

In [37]:
# pipline

In [24]:
from pyspark.ml import Pipeline

In [25]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [26]:
cleaner = data_prep_pipe.fit(data)

In [27]:
clean_data = cleaner.transform(data)

## Task #7: Training and Evaluation!

In [28]:
clean_data = clean_data.select(['label','features'])

In [29]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [30]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [31]:
spam_predictor = nb.fit(training)

In [32]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [33]:
test_results = spam_predictor.transform(testing)

In [34]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-869.88088754084...|[1.0,1.0501107645...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-659.60172456127...|[1.0,5.9896354129...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-461.85542818508...|[1.0,1.0758137246...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-690.09521209562...|[1.0,4.6334498233...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-972.11815221858...|[1.0,6.8174975875...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1312.1464093455...|[1.0,6.0346053104...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-614.81851370625...|[0.99999922223928...|       0.0|
|  0.0|(13424,[0,1,498,5...|[-317.71626690383...|[0.99999999999998...|       0.0|
|  0.0|(13424,[0,2,3,4,6...|[-1284.9489398479...|[1.0,5.4273604468...|       0.0|
|  0.0|(13424,[0

In [39]:
# Evaluate data

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [36]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9295760640004721
